# **WIP**: the model outputs a wrong solution

In [11]:
from typing import Final

BOARD_ROWS: Final[list[int]] = [1, 2, 3, 4, 5, 6, 7, 8]
BOARD_COLS: Final[list[int]] = [1, 2, 3, 4, 5, 6, 7, 8]

In [12]:
class ChessPiece:
    def __init__(self, row: int, col: int) -> None:
        self.row = row
        self.col = col

    def __repr__(self) -> str:
        return f"ChessPiece({self.row}, {self.col})"

    def __str__(self) -> str:
        return self.__repr__()

    def __eq__(self, other: object) -> bool:
        """Check if two chess pieces are equal based on position."""
        if not isinstance(other, ChessPiece):
            return False

        return self.row == other.row and self.col == other.col

    def is_on_same_row(self, other: object) -> bool:
        """Check if two chess pieces are on the same row."""
        if not isinstance(other, ChessPiece):
            return False

        return self.row == other.row

    def is_on_same_col(self, other: object) -> bool:
        """Check if two chess pieces are on the same column."""
        if not isinstance(other, ChessPiece):
            return False

        return self.col == other.col

    def is_on_same_diagonal(self, other: object) -> bool:
        """Check if two chess pieces are on the same diagonal."""
        if not isinstance(other, ChessPiece):
            return False

        return abs(self.row - other.row) == abs(self.col - other.col)

In [13]:
class Chessboard:
    def __init__(self, queens: list[ChessPiece]) -> None:
        self.queens = queens

    def __repr__(self) -> str:
        return f"ChessBoard({self.queens})"

    def __str__(self) -> str:
        return self.__repr__()

    def __eq__(self, other: object) -> bool:
        """Check if two chess boards are equal based on queens positions."""
        if not isinstance(other, Chessboard):
            return False

        return self.queens == other.queens

    def is_goal(self) -> bool:
        """Check if the current chess board is valid."""
        for queen in self.queens:
            for other in self.queens:
                if queen == other:
                    continue

                if (
                    queen.is_on_same_row(other)
                    or queen.is_on_same_col(other)
                    or queen.is_on_same_diagonal(other)
                ):
                    return False

        return True

In [14]:
from typing import Union


class Node:
    def __init__(
        self,
        state: Chessboard,
        parent: Union[Chessboard, None] = None,
    ) -> None:
        self.state = state
        self.parent = parent

    def __repr__(self) -> str:
        return f"SearchState(node={self.state}, parent={self.parent})"

    def __str__(self) -> str:
        return self.__repr__()

    def __eq__(self, other: object) -> bool:
        """Check if two search states are equal based on node."""
        if not isinstance(other, Node):
            return False

        return self.state == other.state

    def is_goal(self):
        """Check if the current search state is valid."""
        return self.state.is_goal()

In [15]:
class StackFrontier:
    def __init__(self) -> None:
        self.frontier: list[Node] = []

    def __repr__(self) -> str:
        return f"StackFrontier(frontier={self.frontier})"

    def __str__(self) -> str:
        return self.__repr__()

    def __len__(self) -> int:
        return len(self.frontier)

    @property
    def is_empty(self) -> bool:
        """Check if the frontier is empty."""
        return len(self.frontier) == 0

    def has_state(self, state: Node) -> bool:
        """Check if the frontier has a state."""
        return any(state == frontier_state for frontier_state in self.frontier)

    def add(self, state: Node) -> None:
        """Add a state to the frontier."""
        self.frontier.append(state)

    def remove(self) -> Node:
        """Remove a state from the frontier."""
        if self.is_empty:
            raise Exception("Empty frontier")

        state = self.frontier.pop()
        return state

    def reconstruct_path(self, state: Node) -> list[Node]:
        """Reconstruct the path from the root node to the current node."""
        path = [state]
        while state.parent is not None:
            path.append(state.parent)
            state = state.parent

        path.reverse()
        return path

In [16]:
class NQueens:
    def __init__(self, size: int) -> None:
        self.size = size
        self.cols = BOARD_COLS[:size]
        self.rows = BOARD_ROWS[:size]
        self._frontier = StackFrontier()

    def neighbors(self, current_node: Node) -> list[Node]:
        """Generate the neighbors of the current node."""

        neighbors: list[Node] = []

        for row in self.rows:
            for col in self.cols:
                new_queens = current_node.state.queens.copy() + [ChessPiece(row, col)]

                tmp_node = Node(
                    state=Chessboard(new_queens),
                    parent=current_node,
                )

                if self._frontier.has_state(tmp_node):
                    continue

                neighbors.append(tmp_node)

        return neighbors

    def solve(self) -> list[Node]:
        initial_state = Node(Chessboard([]))
        self.num_explored = 0
        self._frontier.add(initial_state)
        explored: set[Node] = set()

        while True:
            if self._frontier.is_empty:
                raise Exception("No solution")

            current_node = self._frontier.remove()
            self.num_explored += 1

            if current_node.is_goal():
                return self._frontier.reconstruct_path(current_node)

            explored.add(current_node)

            for _node in self.neighbors(current_node):
                if _node in explored or self._frontier.has_state(_node):
                    continue

                self._frontier.add(_node)

In [17]:
n_queens_problem = NQueens(5)
solution = n_queens_problem.solve()

print(f"Solution: {solution}")

Solution: [SearchState(node=ChessBoard([]), parent=None)]


In [21]:
[1, "A"] == [1, "A"]

True